## Helper functions

In [1]:
import numpy as np
from pynq import MMIO, Overlay, allocate

RAW_TO_VOLTS = 2**-25   # Fixed point to floating point

def uint64_to_signed_int(unsigned):
    """Converts 64-bit unsigned integer to signed integer. By Bit Twiddling Hacks; see
    https://stackoverflow.com/questions/1375897/how-to-get-the-signed-integer-value-of-a-long-in-python.
    """
    unsigned &= (1<<64) - 1  # Keep only the lowest 64 bits.
    return (unsigned ^ (1<<63)) - (1<<63)  # Swap and shift down.

def buffer_to_volts(buffer):
    """Divides integer I and Q values by sample count. Buffer is an array containing a multiple of three
    elements: I value, Q value, count. The I and Q values are divided by count
    and multiplied by a conversion factor to get the unit of volts.
    """
    volts = [
        (
            uint64_to_signed_int(int(buffer[i])          # to Python integer(first entry: unsigned 32-bit integer
            + (int(buffer[i + 1]) << 32))                #     adding second unsigned integer shifted left 32 bits)
            / buffer[i + 2]                              # dividing by third entry (count)
            * RAW_TO_VOLTS                               # scaling to units of volts
        )
        for i in range(0, 12, 3)                         # i = 0, 3, 6, 9
    ]
    return volts

def volts_to_phasors(volts):
    """Interpret the 4 voltage values as phasors
    """
    dut = volts[0] + 1j*volts[1]
    ref = volts[2] + 1j*volts[3]
    rel = dut/ref
    return dut, ref, rel

In [2]:
# Set configuration bits using MMIO

ADC_FREQ = 125_000_000
cyc = lambda x: round(x*ADC_FREQ)
RESET_BIT = 0b1

# Trigger configuration, eg trigger1_conf = TRIG_POS+TRIG_SWEEP
TRIG_POS = 0b0000
TRIG_NEG = 0b0001
TRIG_SWEEP = 0b0010
TRIG_POINTS = 0b0100

mmio_dead_time = MMIO(0x42000000)       # Dead time in ADC samples
mmio_point_time = MMIO(0x42000008)      # Total point time in ADC samples (dead time + accumulation time)
mmio_trigger_conf = MMIO(0x41200000)    # Trigger config
mmio_general_conf = MMIO(0x41200008)    # General config

def set_config(dead_time = 300E-6, point_time = 1E-3, trigger_length = 10E-6, trigger1_conf = 0, trigger2_conf = 0):
#     assert trigger_length <= dead_time <= point_time, "Trigger length should be less than settling time, less than total point time!"
    
    mmio_dead_time.write(0, cyc(dead_time))              # Only one value in register, overwrite completely
    mmio_point_time.write(0, cyc(point_time))            # Idem
    mmio_trigger_conf.write(0, cyc(trigger_length) + (trigger1_conf << 24) + (trigger2_conf << 28))

def read_status():
    return mmio_general_conf.read(0)

def start_acq(dma_recv):
    curr = mmio_general_conf.read(0)
    mmio_general_conf.write(0, curr | RESET_BIT)
    
    # dma request of 16 words to get rid of misformed packet
    buffer = allocate(shape=(16,), dtype=np.uint32)
    dma_recv.transfer(buffer)
    dma_recv.wait()
    del buffer

def stop_acq():
    curr = mmio_general_conf.read(0)
    mmio_general_conf.write(0, curr & ~RESET_BIT)

## Using the VNA

In [3]:
# Program the overlay onto the PL, configure it and get a handle for the DMA

ol = Overlay("/home/xilinx/bit/vna_v1_7.bit")
set_config(dead_time=10E-6, point_time=1000E-6, trigger_length=10E-6, trigger1_conf=TRIG_POS+TRIG_SWEEP, trigger2_conf=TRIG_NEG+TRIG_POINTS)

dma = ol.dma
dma_recv = dma.recvchannel

# start_acq()

In [50]:
set_config(dead_time=10E-6, point_time=1000E-6, trigger_length=10E-6, trigger1_conf=TRIG_POS+TRIG_SWEEP, trigger2_conf=TRIG_NEG+TRIG_POINTS)

start_acq(dma_recv)

In [4]:
# Allocate input buffer and show it's empty

data_size = 12
buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(data_size):
    print(f'0x{format(buffer[i], "02x")}, ', end='')

0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 

In [49]:
# Transfer data and show it (can repeat indefinitely)


# Acquire data
start_acq(dma_recv)
dma_recv.transfer(buffer)
dma_recv.wait()
stop_acq()

# Convert it to human units
volts = buffer_to_volts(buffer)
dut, ref, rel = volts_to_phasors(volts)

# Print results
for i in range(data_size):
    print(f'0x{format(buffer[i], "02x")}, ', end='')
print()
print(volts)
print(f'DUT: {np.abs(dut):.3f}V, {np.angle(dut):.3f}r, ' +
      f'REF: {np.abs(ref):.3f}V, {np.angle(ref):.3f}r, ' +
      f'REL: {np.abs(rel):.3f}, {np.angle(rel):.3f}r')

0xb13d955a, 0xfffffff9, 0x6df, 0x5c397717, 0xfffffffe, 0x6df, 0x9c08c458, 0x01, 0x6df, 0xb770887e, 0xfffffff9, 0x6df, 
[-0.45899927226435805, -0.1193222076700383, 0.1171217299510702, -0.4572371851320359]
DUT: 0.474V, -2.887r, REF: 0.472V, -1.320r, REL: 1.005, -1.567r


In [201]:
cnt = 0
start_acq(dma_recv)
while True:
    dma_recv.transfer(buffer)
    # start_acq()
    dma_recv.wait()
    # stop_acq()
    cnt += 1
    print(f"\r{cnt} transfers (0x{buffer[0]:8x})              ", end='')

2765 transfers (0x63a6626c)              

KeyboardInterrupt: 

## Debugging


In [223]:
# Print DMA status registers
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=0, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

In [157]:
# Print VNA status register
print(read_status())

1


In [245]:
stop_acq()

In [230]:
start_acq(dma_recv)